In [1]:
import os
from ansys.acp.core import launch_acp, Client

In [2]:
SERVER_BIN_PATH = "D:/ANSYSDev/acp_03/bin/release_with_debug_info/lib/acp_grpcserver.exe"
WORKING_DIR = os.getcwd()

In [3]:
server = launch_acp(SERVER_BIN_PATH)

In [4]:
client = Client(server=server)

In [5]:
# TODO: infer from file ending
model = client.import_model(path=os.path.abspath(os.path.join(WORKING_DIR, "class40.cdb")), format="ansys:cdb")
print(model)

In [6]:
#Create the model (unit system is SI)
corecell_81kg_5mm = model.create_fabric(name="Corecell 81kg", thickness=0.005, material=model.materials["1"])
corecell_103kg_10mm = model.create_fabric(name="Corecell 103kg", thickness=0.01, material=model.materials["2"])
eglass_ud_02mm = model.create_fabric(name="eglass UD", thickness=0.0002, material=model.materials["3"])

ros_deck = model.create_rosette( name='ros_deck', 
                                origin=(-5.9334, -0.0481, 1.693) )
ros_hull = model.create_rosette( name='ros_hull', 
                                origin = (-5.3711, -0.0506, -0.2551) )
ros_bulkhead = model.create_rosette( name='ros_bulkhead', 
                                    origin = (-5.622, 0.0022, 0.0847), 
                                    dir1 = (0.0, 1.0, 0.0), 
                                    dir2 = (0.0, 0.0, 1.0) )
ros_keeltower = model.create_rosette( name='ros_keeltower', 
                                     origin = (-6.0699, -0.0502, 0.623), 
                                     dir1 = (0.0, 0.0, 1.0) )

oss_deck = model.create_oriented_selection_set(name='oss_deck', 
                                               orientation_point = (-5.3806, -0.0016, 1.6449), 
                                               orientation_direction = (0., 0., -1.), 
                                               element_sets=[model.element_sets['DECK']], 
                                               rosettes=[ros_deck])
                                                             
oss_hull = model.create_oriented_selection_set(name='oss_hull', 
                                               orientation_point = (-5.12, 0.1949, -0.2487), 
                                               orientation_direction = (0., 0., 1.), 
                                               element_sets=[model.element_sets['HULL_ALL']], 
                                               rosettes=[ros_hull])                                                     

oss_bulkhead = model.create_oriented_selection_set(name='oss_bulkhead', 
                                                   orientation_point = (-5.622, -0.0465, -0.094), 
                                                   orientation_direction = (1., 0., 0.), 
                                                   element_sets=[model.element_sets['BULKHEAD_ALL']], 
                                                   rosettes=[ros_bulkhead])

esets = [model.element_sets['KEELTOWER_AFT'],
         model.element_sets['KEELTOWER_FRONT'],
         model.element_sets['KEELTOWER_PORT'],
         model.element_sets['KEELTOWER_STB']
        ]

oss_keeltower = model.create_oriented_selection_set(name='oss_keeltower', 
                                                    orientation_point = (-6.1019, 0.0001, 1.162), 
                                                    orientation_direction = (-1., 0., 0.), 
                                                    element_sets=esets, 
                                                    rosettes=[ros_keeltower])

In [7]:
def add_ply(mg, name, ply_material, angle, oss):
    #todo: pass angle
    return mg.create_modeling_ply(name=name,
                                  ply_material=ply_material,
                                  oriented_selection_sets = oss)

angles = [-90., -60., -45. -30., 0., 0., 30., 45., 60., 90.]

#define plies for the HULL, DECK and BULKHEAD
for mg_name in ['hull', 'deck', 'bulkhead']:
    mg = model.create_modeling_group(name=mg_name)
    oss_list = [model.oriented_selection_sets["oss_"+mg_name]]
    
    for angle in angles:
        add_ply(mg, "eglass_ud_02mm_"+str(angle), eglass_ud_02mm, angle, oss_list)
        
    add_ply(mg, "corecell_103kg_10mm", corecell_103kg_10mm, 0., oss_list)
    
    for angle in angles:
        add_ply(mg, "eglass_ud_02mm_"+str(angle), eglass_ud_02mm, angle, oss_list)
    
#add plies to the keeltower
mg = model.create_modeling_group(name='keeltower')
oss_list = [model.oriented_selection_sets["oss_keeltower"]]
for angle in angles:
    add_ply(mg, "eglass_ud_02mm_"+str(angle), eglass_ud_02mm, angle, oss_list)
        
add_ply(mg, "corecell_81kg_5mm", corecell_81kg_5mm, 0., oss_list)
    
for angle in angles:
    add_ply(mg, "eglass_ud_02mm_"+str(angle), eglass_ud_02mm, angle, oss_list)

In [8]:
print(len(model.modeling_groups))
print(len(model.modeling_groups["hull"].modeling_plies))
print(len(model.modeling_groups["deck"].modeling_plies))
print(len(model.modeling_groups["bulkhead"].modeling_plies))
print(len(model.modeling_groups["keeltower"].modeling_plies))

4
19
19
19
19


In [9]:
model.update()
model.save(os.path.abspath("./results/class40.acph5"), save_cache=True)

In [11]:
model.save_analysis_model(os.path.abspath("./results/class40_analysis_model.cdb"))

In [12]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

In [13]:
mapdl = launch_mapdl(additional_switches="-smp")

In [14]:
mapdl.input(os.path.join(WORKING_DIR,"results","class40_analysis_model.cdb"))

'\n /INPUT FILE= D:\\ANSYSDev\\pyacp-private\\examples\\class40\\results\\class40_analysis_model.cdb  LINE=       0\n ANSYS RELEASE 11.0    UP20070125       16:39:41    03/10/2009\n\n *** ANSYS - ENGINEERING ANALYSIS SYSTEM  RELEASE 2022 R1          22.1BETA ***\n Ansys Mechanical Enterprise                       \n 00000000  VERSION=WINDOWS x64   10:53:58  JUN 27, 2022 CP=      0.172\n\n                                                                               \n\n\n\n ** WARNING: PRE-RELEASE VERSION OF ANSYS 22.1BETA\n  ANSYS,INC TESTING IS NOT COMPLETE - CHECK RESULTS CAREFULLY **\n\n          ***** ANSYS ANALYSIS DEFINITION (PREP7) *****\n\n\n ***** ROUTINE COMPLETED *****  CP =         0.312\n\n\n'

In [15]:
mapdl.allsel()
mapdl.slashsolu()
mapdl.solve()

*****  ANSYS SOLVE    COMMAND  *****

 *** NOTE ***                            CP =       0.531   TIME= 10:53:59
 There is no title defined for this analysis.                            

 *** SELECTION OF ELEMENT TECHNOLOGIES FOR APPLICABLE ELEMENTS ***
                ---GIVE SUGGESTIONS ONLY---

 ELEMENT TYPE         1 IS SHELL181. IT IS ASSOCIATED WITH ELASTOPLASTIC 
 MATERIALS ONLY. KEYOPT(8) IS ALREADY SET AS SUGGESTED. KEYOPT(3)=2 
 IS SUGGESTED FOR HIGHER ACCURACY OF MEMBRANE STRESSES; OTHERWISE, 
 KEYOPT(3)=0 IS SUGGESTED.

 ELEMENT TYPE         2 IS BEAM188 . KEYOPT(1)=1 IS SUGGESTED FOR NON-CIRCULAR CROSS
 SECTIONS AND KEYOPT(3)=2 IS ALWAYS SUGGESTED.

 ELEMENT TYPE         2 IS BEAM188 . KEYOPT(15) IS ALREADY SET AS SUGGESTED.

 ELEMENT TYPE         3 IS SHELL181. IT IS ASSOCIATED WITH ELASTOPLASTIC 
 MATERIALS ONLY. KEYOPT(8) IS ALREADY SET AS SUGGESTED. KEYOPT(3)=2 
 IS SUGGESTED FOR HIGHER ACCURACY OF MEMBRANE STRESSES; OTHERWISE, 
 KEYOPT(3)=0 IS SUGGESTED.



 *** ANSY

In [16]:
mapdl.post1()
mapdl.set("last")
mapdl.post_processing.plot_nodal_displacement(component="NORM")

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [20]:
mapdl.post_processing.plot_element_stress("X")

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)